In [1]:
#!usr/bin/python
# -*- coding: utf-8 -*-
import numpy as np
import scipy.io as sio
import time
import random
# import tensorflow as tf
import numpy as np
import scipy
import scipy.sparse as sp
from sklearn.cluster import KMeans
from mGCN_Toolbox.model.MvAGC.metrics import clustering_metrics
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import normalize
from time import *
import warnings
warnings.filterwarnings("ignore")
# warnings.simplefilter('error',ComplexWarning)
from mGCN_Toolbox.utils.dataset import dataset
from mGCN_Toolbox.utils.process import * 




def FGC_cora_modified(X, av, gnd, a, k, ind):
        # Store some variables
    gama=-1
    nada = [1, 1]
    X_hat_list=[]
    X_hat_anchor_list=[]
    A_hat_list=[]
    final = []
    for i in range(2):
        A=av[i]
        N = X.shape[0]
        # print("N = {}".format(N))
        Im = np.eye(len(ind))
        In = np.eye(N)
        if sp.issparse(X):
            X = X.todense()

        # Normalize A
        A = A + In
        D = np.sum(A, axis=1)
        D = np.power(D, -0.5)
        D[np.isinf(D)] = 0
        D = np.diagflat(D)
        A = D.dot(A).dot(D)

        # Get filter G
        Ls = In - A
        G = In - 0.2 * Ls #0.5 #0.8
        G_ = In
        X_hat = X
        for i in range(k):
            # G_ = G_.dot(G)
            X_hat = G.dot(X_hat)
        X_hat_list.append(X_hat)
        A_hat = A[ind]  # (m,n)
        A_hat_list.append(A_hat)
        X_hat_anchor_list.append(X_hat[ind])
    begin_time = time()
    # Set the order of filter
    for t in range(20):
        tmp1=0
        tmp2=0
        for i in range(2):
            tmp1 =tmp1+nada[i]*(X_hat_anchor_list[i].dot(X_hat_anchor_list[i].T) + a * Im)
        for i in range(2):
            tmp2 = tmp2+nada[i]*(X_hat_anchor_list[i].dot(X_hat_list[i].T) + a * A_hat_list[i])
        S = np.linalg.inv(tmp1).dot(tmp2)
        for i in range(2):
            nada[i] = (-((np.linalg.norm(X_hat_list[i].T - (X_hat_anchor_list[i].T).dot(S))) ** 2 + a * (np.linalg.norm(S - A_hat_list[i])) ** 2) / (gama)) ** (1 / (gama - 1))
            print("nada value")
            print(nada[i])
        # res=0
        # for j in range(2):
        #     res = res + nada[j] * ((np.linalg.norm(X_hat_list[i].T - (X_hat_anchor_list[i].T).dot(S))) ** 2 + a * (np.linalg.norm(S - A_hat_list[i])) ** 2) + (nada[j]) ** (gama)
        # final.append(res)
        # print(res)
    # sio.savemat("a.mat", {'res': final})
    return S, begin_time


def main(X, av, gnd, m, a, k, ind):

    N = X.shape[0]
    begin_time_filter = time()
    types = len(np.unique(gnd))
    S, begin_time = FGC_cora_modified(X, av, gnd, a, k, ind)
    D = np.sum(S, axis=1)
    D = np.power(D, -0.5)
    D[np.isinf(D)] = 0
    D[np.isnan(D)] = 0
    D = np.diagflat(D)  # (m,m)

    S_hat = D.dot(S)  # (m,n)

    S_hat_tmp = S_hat.dot(S_hat.T)  # (m,m)
    S_hat_tmp[np.isinf(S_hat_tmp)] = 0
    S_hat_tmp[np.isnan(S_hat_tmp)] = 0
    # sigma, E = scipy.linalg.eig(S_hat_tmp)
    E, sigma, v = sp.linalg.svds(S_hat_tmp, k=types, which='LM')
    sigma = sigma.T
    sigma = np.power(sigma, -0.5)
    sigma[np.isinf(sigma)] = 0
    sigma[np.isnan(sigma)] = 0
    sigma = np.diagflat(sigma)
    C_hat = (sigma.dot(E.T)).dot(S_hat)
    C_hat[np.isinf(C_hat)] = 0
    C_hat[np.isnan(C_hat)] = 0
    C_hat = C_hat.astype(float)
    print("print shape of C_hat: ", C_hat.shape)
    kmeans = KMeans(n_clusters=types, random_state=37).fit(C_hat.T)

    predict_labels = kmeans.predict(C_hat.T)

    cm = clustering_metrics(gnd, predict_labels)
    ac, nm, f1,adj,sim = cm.evaluationClusterModelFromLabel(m,a,k)
    end_time = time()
    tot_time = end_time - begin_time
    tot_time_filter = end_time - begin_time_filter
    return ac, nm, f1,adj, tot_time, tot_time_filter,sim


def lower_bound(p, rd):
    l = 0
    r = len(p) - 1
    while(l < r):
        # print("rd = {}, l = {}, r= {}".format(rd, l, r))
        mid = (l + r) // 2
        if(p[mid] > rd):
            r = mid
        else:
            l = mid + 1
    # print("rd = {}, l = {}, r= {}".format(rd, l, r))
    return l


def node_sampling(A, m, alpha):
    D = np.sum(A[0], axis=1).flatten()+np.sum(A[1], axis=1).flatten()

    if(len(np.shape(D)) > 1):
        D = D.A[0]
        print(1)

    D = D**alpha
    D=D/10000
    print(D)
    tot = np.sum(D)
    print(tot)
    p = D / tot
    print(p)
    for i in range(len(p) - 1):
        p[i + 1] = p[i + 1] + p[i]
    print(p)
    ind = []
    vis = [0] * len(D)
    while(m):
        while(1):
            rd = np.random.rand()
            pos = lower_bound(p, rd)
            if(vis[pos] == 1):
                continue
            else:
                vis[pos] = 1
                ind.append(pos)
                m = m - 1
                break
    return ind


def func(X, A, gnd):
    m_init_list = [60] #anchor numbers
    a_list = [5,15] #second term
    k_init_list = [2] #juanjijieshu
    f_alpha_init_list = [4] #important node
    k_list = []
    aa_list = []
    i_list = []
    ac_list = []
    nm_list = []
    f1_list = []
    adj_list=[]
    tm_list = []
    tm_list_filter = []
    f_alpha_list = []

    N = X.shape[0]
    tot_test = 1
    ac_max = 0.0
    xia = 0
    tot = 0

    # print(node_sampling(A, 20))
    for k in k_init_list:
        for i in m_init_list:
            # print("now k = {}, now m = {}".format(k, i))
            for alpha in f_alpha_init_list:
                ind = node_sampling(A, i, alpha)
                ac_mean = 0
                nm_mean = 0
                f1_mean = 0
                adj_mean=0
                tm_mean = 0
                for a in a_list:
                    # continue
                    acc, nmm, f11,adj, tm, tm_filter,sim = main(
                        X, A, gnd, i, a, k, ind)
                    print("m = {},k = {}, f_alpha = {},a  ={}, ac = {}, nmi = {}, f1 = {},adj={}, tm = {}, tm_filter = {}".format(
                        i, k, alpha, a, acc, nmm, f11,adj, tm, tm_filter))
                    if(ac_mean < acc):
                        ac_mean = acc
                        nm_mean = nmm
                        f1_mean = f11
                        adj_mean=adj
                        tm_mean = tm
                        tm_mean_filter = tm_filter
                    i_list.append(i)
                    k_list.append(k)
                    aa_list.append(a)
                    f_alpha_list.append(alpha)
                    ac_list.append(ac_mean)
                    nm_list.append(nm_mean)
                    f1_list.append(f1_mean)
                    adj_list.append(adj_mean)
                    tm_list.append(tm_mean)
                    tm_list_filter.append(tm_mean_filter)
                print("m = {}, k ={},f_alpha = {}, ac_mean = {},nm_mean = {},f1_mean = {},adj_mean={},tm_mean = {},tm_mean_filter = {}\n".format(
                    i, k, alpha, ac_mean, nm_mean, f1_mean,adj_mean, tm_mean, tm_mean_filter))

                if(ac_mean > ac_max):
                    xia = tot
                    ac_max = ac_mean

                tot += 1

    for i in range(len(i_list)):
        print("m = {},k = {},f_alpha = {}, ac_mean = {}, nm_mean = {}, f1_mean = {},adj_mean={},tm_mean = {},tm_mean_filter ={}".format(
            i_list[i], k_list[i], f_alpha_list[i], ac_list[i], nm_list[i], f1_list[i],adj_list[i], tm_list[i], tm_list_filter[i]))
    print("the best result is ")
    print("m = {},k = {},f_alpha = {}, ac_mean = {}, nm_mean = {}, f1_mean = {},adj_mean={},tm_mean = {},tm_mean_filter = {}".format(
        i_list[xia], k_list[xia], f_alpha_list[xia], ac_list[xia], nm_list[xia], f1_list[xia],adj_list[xia], tm_list[xia], tm_list_filter[xia]))
    return i_list[xia], k_list[xia], f_alpha_list[xia], ac_list[xia], nm_list[xia], f1_list[xia],adj_list[xia], tm_list[xia], tm_list_filter[xia]


if __name__ == '__main__':
    dataname = 'IMDB3025'#'ACM3025'#'AMAZON3025'
    s = dataset(dataname)
    # number of epoch
    tt = 100
    print(tt)
    m_best_list = []
    k_best_list = []
    f_alpha_best_list = []
    ac_best_list = []
    nm_best_list = []
    f1_best_list = []
    adj_best_list = []
    tm_best_list = []
    tm_filter_best_list = []
    for i in range(tt):
        nowm, nowk, nowf, nowac, nownm, nowf1,nowadj, nowtm, nowtmf = func(s.X, s.av, s.gnd)
        m_best_list.append(nowm)
        k_best_list.append(nowk)
        f_alpha_best_list.append(nowf)
        ac_best_list.append(nowac)
        nm_best_list.append(nownm)
        f1_best_list.append(nowf1)
        adj_best_list.append(nowadj)
        tm_best_list.append(nowtm)
        tm_filter_best_list.append(nowtmf)
        print("iteration {}, m = {}, k = {}, f_alpha = {}, ac = {}, nm = {}, f1 = {},adj={}, tm = {}, tm_filter = {}".format(
            i + 1, nowm, nowk, nowf, nowac, nownm, nowf1,nowadj, nowtm, nowtmf))
    for i in range(len(ac_best_list)):
        print("iteration {}, m = {}, k = {}, f_alpha = {}, ac = {}, nm = {}, f1 = {},adj={}, tm = {}, tm_filter = {}".format(
            i + 1, m_best_list[i], k_best_list[i], f_alpha_best_list[i], ac_best_list[i], nm_best_list[i], f1_best_list[i],adj_best_list[i], tm_best_list[i], tm_filter_best_list[i]))
    print("ac_mean = {}, ac_std = {}".format(
        np.mean(ac_best_list), np.std(ac_best_list, ddof=1)))
    print("nm_mean = {}, nm_std = {}".format(
        np.mean(nm_best_list), np.std(nm_best_list, ddof=1)))
    print("f1_mean = {}, f1_std = {}".format(
        np.mean(f1_best_list), np.std(f1_best_list, ddof=1)))
    print("adj_mean = {}, adj_std = {}".format(
        np.mean(adj_best_list), np.std(adj_best_list, ddof=1)))
    print("tm_mean = {}, tm_std = {}".format(
        np.mean(tm_best_list), np.std(tm_best_list, ddof=1)))
    print("tmf_mean = {}, tmf_std = {}".format(
        np.mean(tm_filter_best_list), np.std(tm_filter_best_list, ddof=1)))




100
[1.944810e+01 2.825761e+02 8.352100e+00 ... 1.600000e-03 1.000000e+00
 2.560000e-02]
753312.4588
[2.58167773e-05 3.75111412e-04 1.10871656e-05 ... 2.12395266e-09
 1.32747041e-06 3.39832425e-08]
[2.58167773e-05 4.00928189e-04 4.12015355e-04 ... 9.99998639e-01
 9.99999966e-01 1.00000000e+00]
nada value
0.0023528202127914865
nada value
0.0027486172460939777
nada value
0.0023525139518402326
nada value
0.002749069080145868
nada value
0.0023525133434470122
nada value
0.0027490699110589256
nada value
0.0023525133422884407
nada value
0.0027490699126410147
nada value
0.0023525133422862303
nada value
0.002749069912644029
nada value
0.0023525133422862255
nada value
0.0027490699126440357
nada value
0.0023525133422862255
nada value
0.0027490699126440352
nada value
0.0023525133422862255
nada value
0.0027490699126440357
nada value
0.0023525133422862255
nada value
0.0027490699126440352
nada value
0.0023525133422862255
nada value
0.0027490699126440357
nada value
0.0023525133422862255
nada value
0.0

nada value
0.002356667497355664
nada value
0.002753688391700442
nada value
0.0023563863166132164
nada value
0.002754103428298993
nada value
0.0023563858053067895
nada value
0.002754104126865403
nada value
0.0023563858044155657
nada value
0.0027541041280828705
nada value
0.002356385804414014
nada value
0.002754104128084989
nada value
0.0023563858044140106
nada value
0.0027541041280849925
nada value
0.0023563858044140106
nada value
0.0027541041280849925
nada value
0.0023563858044140106
nada value
0.0027541041280849925
nada value
0.0023563858044140106
nada value
0.0027541041280849925
nada value
0.0023563858044140106
nada value
0.0027541041280849925
nada value
0.0023563858044140106
nada value
0.0027541041280849925
nada value
0.0023563858044140106
nada value
0.0027541041280849925
nada value
0.0023563858044140106
nada value
0.0027541041280849925
nada value
0.0023563858044140106
nada value
0.0027541041280849925
nada value
0.0023563858044140106
nada value
0.0027541041280849925
nada value
0.002

nada value
0.0023543371204814523
nada value
0.002751553968967477
nada value
0.002354053237190584
nada value
0.00275197322329366
nada value
0.002354052716219938
nada value
0.0027519739353734926
nada value
0.0023540527153035713
nada value
0.002751973936625842
nada value
0.002354052715301957
nada value
0.002751973936628046
nada value
0.0023540527153019545
nada value
0.00275197393662805
nada value
0.0023540527153019545
nada value
0.00275197393662805
nada value
0.0023540527153019545
nada value
0.00275197393662805
nada value
0.0023540527153019545
nada value
0.00275197393662805
nada value
0.0023540527153019545
nada value
0.00275197393662805
nada value
0.0023540527153019545
nada value
0.00275197393662805
nada value
0.0023540527153019545
nada value
0.00275197393662805
nada value
0.0023540527153019545
nada value
0.00275197393662805
nada value
0.0023540527153019545
nada value
0.00275197393662805
nada value
0.0023540527153019545
nada value
0.00275197393662805
nada value
0.0023540527153019545
nada 

nada value
0.002358028801760934
nada value
0.002755434590755227
nada value
0.002357739892239984
nada value
0.002755861101207823
nada value
0.0023577393529588676
nada value
0.0027558618380909587
nada value
0.002357739351993988
nada value
0.0027558618394092063
nada value
0.00235773935199226
nada value
0.002755861839411566
nada value
0.002357739351992257
nada value
0.0027558618394115707
nada value
0.002357739351992257
nada value
0.0027558618394115707
nada value
0.002357739351992257
nada value
0.0027558618394115707
nada value
0.002357739351992257
nada value
0.0027558618394115707
nada value
0.002357739351992257
nada value
0.0027558618394115707
nada value
0.002357739351992257
nada value
0.0027558618394115707
nada value
0.002357739351992257
nada value
0.0027558618394115707
nada value
0.002357739351992257
nada value
0.0027558618394115707
nada value
0.002357739351992257
nada value
0.0027558618394115707
nada value
0.002357739351992257
nada value
0.0027558618394115707
nada value
0.002357739351992

nada value
0.0023538409216555837
nada value
0.0027511506925606945
nada value
0.002353561238451913
nada value
0.0027515637960791573
nada value
0.002353560732308741
nada value
0.0027515644879742697
nada value
0.0023535607314308102
nada value
0.002751564489174237
nada value
0.0023535607314292893
nada value
0.00275156448917632
nada value
0.0023535607314292867
nada value
0.002751564489176323
nada value
0.0023535607314292867
nada value
0.002751564489176323
nada value
0.0023535607314292867
nada value
0.002751564489176323
nada value
0.0023535607314292867
nada value
0.002751564489176323
nada value
0.0023535607314292867
nada value
0.002751564489176323
nada value
0.0023535607314292867
nada value
0.002751564489176323
nada value
0.0023535607314292867
nada value
0.002751564489176323
nada value
0.0023535607314292867
nada value
0.002751564489176323
nada value
0.0023535607314292867
nada value
0.002751564489176323
nada value
0.0023535607314292867
nada value
0.002751564489176323
nada value
0.002353560731

...

nada value
0.002352869625242302
nada value
0.0027514741579800087
nada value
0.0023525708439106965
nada value
0.002751916089753248
nada value
0.0023525702687573246
nada value
0.0027519168768559794
nada value
0.002352570267696317
nada value
0.0027519168783077695
nada value
0.002352570267694362
nada value
0.002751916878310445
nada value
0.0023525702676943576
nada value
0.00275191687831045
nada value
0.0023525702676943576
nada value
0.00275191687831045
nada value
0.0023525702676943576
nada value
0.00275191687831045
nada value
0.0023525702676943576
nada value
0.00275191687831045
nada value
0.0023525702676943576
nada value
0.00275191687831045
nada value
0.0023525702676943576
nada value
0.00275191687831045
nada value
0.0023525702676943576
nada value
0.00275191687831045
nada value
0.0023525702676943576
nada value
0.00275191687831045
nada value
0.0023525702676943576
nada value
0.00275191687831045
nada value
0.0023525702676943576
nada value
0.00275191687831045
nada value
0.0023525702676943576
na

nada value
0.0023560863273738544
nada value
0.0027532867127654224
nada value
0.0023558107301074336
nada value
0.0027536936098563275
nada value
0.002355810239185874
nada value
0.0027536942806984685
nada value
0.002355810238347684
nada value
0.0027536942818436987
nada value
0.0023558102383462523
nada value
0.002753694281845652
nada value
0.0023558102383462497
nada value
0.002753694281845655
nada value
0.00235581023834625
nada value
0.002753694281845655
nada value
0.00235581023834625
nada value
0.0027536942818456554
nada value
0.00235581023834625
nada value
0.0027536942818456554
nada value
0.00235581023834625
nada value
0.0027536942818456554
nada value
0.00235581023834625
nada value
0.0027536942818456554
nada value
0.00235581023834625
nada value
0.0027536942818456554
nada value
0.00235581023834625
nada value
0.0027536942818456554
nada value
0.00235581023834625
nada value
0.0027536942818456554
nada value
0.00235581023834625
nada value
0.0027536942818456554
nada value
0.00235581023834625
na

nada value
0.0023570688163557867
nada value
0.0027553652222918894
nada value
0.0023567853164382993
nada value
0.0027557841516927225
nada value
0.0023567847984135634
nada value
0.0027557848600629326
nada value
0.002356784797506377
nada value
0.0027557848613032894
nada value
0.002356784797504789
nada value
0.0027557848613054617
nada value
0.0023567847975047864
nada value
0.0027557848613054656
nada value
0.0023567847975047864
nada value
0.0027557848613054656
nada value
0.0023567847975047864
nada value
0.0027557848613054656
nada value
0.0023567847975047864
nada value
0.0027557848613054656
nada value
0.0023567847975047864
nada value
0.0027557848613054656
nada value
0.0023567847975047864
nada value
0.0027557848613054656
nada value
0.0023567847975047864
nada value
0.0027557848613054656
nada value
0.0023567847975047864
nada value
0.0027557848613054656
nada value
0.0023567847975047864
nada value
0.0027557848613054656
nada value
0.0023567847975047864
nada value
0.0027557848613054656
nada value
0

nada value
0.0023562880767131087
nada value
0.002752858714621893
nada value
0.0023560090347889816
nada value
0.0027532704416181105
nada value
0.002356008530447674
nada value
0.002753271130471614
nada value
0.002356008529573878
nada value
0.0027532711316649336
nada value
0.0023560085295723627
nada value
0.002753271131667003
nada value
0.00235600852957236
nada value
0.002753271131667006
nada value
0.00235600852957236
nada value
0.002753271131667006
nada value
0.00235600852957236
nada value
0.002753271131667006
nada value
0.00235600852957236
nada value
0.002753271131667006
nada value
0.00235600852957236
nada value
0.002753271131667006
nada value
0.00235600852957236
nada value
0.002753271131667006
nada value
0.00235600852957236
nada value
0.002753271131667006
nada value
0.00235600852957236
nada value
0.002753271131667006
nada value
0.00235600852957236
nada value
0.002753271131667006
nada value
0.00235600852957236
nada value
0.002753271131667006
nada value
0.00235600852957236
nada value
0.0

KeyboardInterrupt: 

In [ ]:
parser.add_argument('--embedder', nargs='?', default='HDMI', help='HDMI or HDI')
    parser.add_argument('--hid_units', type=int, default=128, help='hidden dimension')
    parser.add_argument('--same_discriminator', type=bool, default=False,
                        help='whether to use the same discriminator for the layers and fusion module')

    parser.add_argument('--dataset', nargs='?', default='imdb')
    parser.add_argument('--sc', type=float, default=3.0, help='GCN self connection')
    parser.add_argument('--sparse', type=bool, default=True, help='sparse adjacency matrix')

In [ ]:
model = modeler(self.args.ft_size, self.args.hid_units, len(adj_list), self.args.same_discriminator)\
            .to(self.args.device)

In [ ]:
class modeler(nn.Module):
    def __init__(self, ft_size, hid_units, n_networks, same_discriminator=True):
        super(modeler, self).__init__()
        self.gcn_list = nn.ModuleList([GCN(ft_size, hid_units) for _ in range(n_networks)])
        self.w_list = nn.ModuleList([nn.Linear(hid_units, hid_units, bias=False) for _ in range(n_networks)])
        self.y_list = nn.ModuleList([nn.Linear(hid_units, 1) for _ in range(n_networks)])

In [ ]:
self.att_act1 = nn.Tanh()
self.att_act2 = nn.Softmax(dim=-1)

In [ ]:
def embed(self, seq, adj_list, sparse):
        h_1_list = []
        for i, adj in enumerate(adj_list):
            h_1 = torch.squeeze(self.gcn_list[i](seq, adj, sparse))
            h_1_list.append(h_1)
        h = self.combine_att(h_1_list)
        return h.detach()

In [ ]:
embeds = model.embed(features, adj_list, self.args.sparse)
        macro_f1s, micro_f1s, k1 = evaluate(embeds, self.idx_train, self.idx_val, self.idx_test, self.labels,)
        return macro_f1s, micro_f1s, k1

In [ ]:
def combine_att(self, h_list):
        h_combine_list = []
        for i, h in enumerate(h_list):
            h = self.w_list[i](h)
            h = self.y_list[i](h)
            h_combine_list.append(h)
        score = torch.cat(h_combine_list, -1)
        score = self.att_act1(score)
        score = self.att_act2(score)
        score = torch.unsqueeze(score, -1)
        h = torch.stack(h_list, dim=1)
        h = score * h
        h = torch.sum(h, dim=1)
        return h